In [4]:
from batter import MABFESystem
from batter.input_process import get_configure_from_file, parse_input_file

In [5]:
protein_file = 'data/7T2G_mp/protein_input.pdb'
system_file = 'data/7T2G_mp/system_input.pdb'
ligand_files = ['data/7T2G_mp/ligand_MP.pdb',
                'data/7T2G_mp/ligand_MP2.pdb',
                'data/7T2G_mp/ligand_DAMGO.pdb']
#system_inpcrd = 'data/7T2G_mp/system_input.inpcrd'
equilibrated_rst = 'data/7T2G_mp/equilibrated.rst'

input_file = 'data/input_files/abfe.in'

In [ ]:
output_folder = 'test/abfe'
system = MABFESystem(folder=output_folder)

In [ ]:
# check available input of create_system
system.create_system?

In [ ]:
system.create_system(
            system_name='7T2G',
            protein_input=protein_file,
            system_topology=system_file,
            system_coordinate=equilibrated_rst,
            ligand_paths=ligand_files,
#            overwrite=True,
            lipid_mol=['POPC'])

In [ ]:
import MDAnalysis as mda

u_prot = mda.Universe(f'{output_folder}/all-poses/reference.pdb')
u_lig = mda.Universe(f'{output_folder}/all-poses/pose0.pdb')
u_merge = mda.Merge(u_prot.atoms, u_lig.atoms)

P1_atom = u_merge.select_atoms('name CA and resid 149')
P2_atom = u_merge.select_atoms('name CA and resid 119')
P3_atom = u_merge.select_atoms('name CA and resid 328')
if P1_atom.n_atoms != 1 or P2_atom.n_atoms != 1 or P3_atom.n_atoms != 1:
    raise ValueError('Error: more than one atom selected')

lig_atom = u_merge.select_atoms(f'resname {u_lig.atoms.resnames[0]} and name C12')

# get ll_x,y,z distances
r_vect = lig_atom.center_of_mass() - P1_atom.positions
print(f'l1_x: {r_vect[0][0]:.2f}')
print(f'l1_y: {r_vect[0][1]:.2f}')
print(f'l1_z: {r_vect[0][2]:.2f}')

Modify the `test/abfe/abfe.in` file to include the following lines based on the output of the above script:
    
```bash
    l1_x: xxx
    l1_y: xxx
    l1_z: xxx
```

In [ ]:
# Prepare the system for the equilibration stage
system.prepare(
    stage='equil',
    input_file=input_file,
    overwrite=True,
)

In [ ]:
# Submit the equilibration job to SLURM
system.submit(stage='equil')

Wait for the equilbration to finish.

In [ ]:
# Prepare the system for the free energy calculation
system.prepare(
    stage='fe',
    input_file=input_file,
#    overwrite=True,
)

In [ ]:
# Submit the equilibration job to SLURM
system.submit(stage='fe')

Wait for the production to finish.

In [ ]:
system.analysis()